In [4]:
from langchain.llms import Clarifai
from langchain.agents import load_tools, initialize_agent
from langchain.agents import AgentType
from langchain.tools import AIPluginTool
from langchain.text_splitter import TokenTextSplitter
from langchain.chains import APIChain
import requests

USER_ID = 'openai'
APP_ID = 'chat-completion'
MODEL_ID = 'GPT-4' 
MODEL_VERSION_ID = 'ad16eda6ac054796bf9f348ab6733c72'

tool = AIPluginTool.from_plugin_url("http://localhost:8000/.well-known/ai-plugin.json")


clarifai_llm = Clarifai(user_id=USER_ID,app_id=APP_ID, model_id=MODEL_ID, model_version_id=MODEL_VERSION_ID)

# Used to load the plugin
tools = load_tools(["requests_all"])

tools += [tool]

agent_chain = initialize_agent(tools, clarifai_llm, verbose=True, agent_chain=AgentType.SELF_ASK_WITH_SEARCH)
agent_chain.run("How many home runs do the Yankees have for the 2023 season?")




> Entering new AgentExecutor chain...
In order to find out the number of home runs the Yankees have for the 2023 season, I would need to access the MLB Stats API. We don't have the exact information regarding how to use this API, so we should use the Major_League_Baseball_Statistics_Plugin to get the OpenAPI specification and usage guide for this API.
Action: Major_League_Baseball_Statistics_Plugin
Action Input: {}
Observation: Usage Guide: Plugin for retrieving up to date baseball statistics for teams and players across the MLB. This plugin should be used for retrieving any baseball-related information that is current beyond ChatGPT's cutoff date of September 2021.

OpenAPI Spec: {'openapi': '3.1.0', 'info': {'title': 'MLB Stats', 'description': 'Get current and historical statistics for MLB players, teams, games, and seasons.', 'version': 'v1'}, 'servers': [{'url': 'http://localhost:8000/'}], 'paths': {'/standings': {'get': {'operationId': 'getStandings', 'summary': 'Retrieves the 

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:25                                                                                   │
│                                                                                                  │
│   22 tools += [tool]                                                                             │
│   23                                                                                             │
│   24 agent_chain = initialize_agent(tools, clarifai_llm, verbose=True, agent_chain=AgentType.    │
│ ❱ 25 agent_chain.run("How many home runs do the Yankees have for the 2023 season?")              │
│   26                                                                                             │
│                                                                                                  │
│ c:\Users\joefo\dev\mlb-stats-chatbot\mlb-env\lib\site-packages\langchain\chains\base.py:487 in   │
│ run                                                                                              │
│                                                                                                  │
│   484 │   │   if args and not kwargs:                                                            │
│   485 │   │   │   if len(args) != 1:                                                             │
│   486 │   │   │   │   raise ValueError("`run` supports only one positional argument.")           │
│ ❱ 487 │   │   │   return self(args[0], callbacks=callbacks, tags=tags, metadata=metadata)[       │
│   488 │   │   │   │   _output_key                                                                │
│   489 │   │   │   ]                                                                              │
│   490                                                                                            │
│                                                                                                  │
│ c:\Users\joefo\dev\mlb-stats-chatbot\mlb-env\lib\site-packages\langchain\chains\base.py:292 in   │
│ __call__                                                                                         │
│                                                                                                  │
│   289 │   │   │   )                                                                              │
│   290 │   │   except (KeyboardInterrupt, Exception) as e:                                        │
│   291 │   │   │   run_manager.on_chain_error(e)                                                  │
│ ❱ 292 │   │   │   raise e                                                                        │
│   293 │   │   run_manager.on_chain_end(outputs)                                                  │
│   294 │   │   final_outputs: Dict[str, Any] = self.prep_outputs(                                 │
│   295 │   │   │   inputs, outputs, return_only_outputs                                           │
│                                                                                                  │
│ c:\Users\joefo\dev\mlb-stats-chatbot\mlb-env\lib\site-packages\langchain\chains\base.py:286 in   │
│ __call__                                                                                         │
│                                                                                                  │
│   283 │   │   )                                                                                  │
│   284 │   │   try:                                                                               │
│   285 │   │   │   outputs = (                                                                    │
│ ❱ 286 │   │   │   │   self._call(inputs, run_manager=run_manager)                                │
│   287 │   │   │   │   if new_arg_supported                                                       │
│   288 │   │   │   │   else self._call(inputs)              

In [7]:
from langchain.llms import Clarifai
from langchain.tools import AIPluginTool
from langchain.text_splitter import TokenTextSplitter
from langchain.chains import APIChain, SimpleSequentialChain, LLMChain
import requests
from datetime import datetime
from langchain.memory import ConversationBufferMemory

from langchain.prompts import PromptTemplate

USER_ID = 'openai'
APP_ID = 'chat-completion'
MODEL_ID = 'GPT-4' 
MODEL_VERSION_ID = 'ad16eda6ac054796bf9f348ab6733c72'

current_year = datetime.now().year

baseball_api = requests.get("http://localhost:8000/openapi.yaml").text

MLB_PROMPT_TEMPLATE = """You are given the below Major League Baseball (MLB) API Documentation:
{api_docs}
Using this documentation, generate the full API url to call for answering the user question.
The current year is 2023. You should make sure to include the required parameters when calling the various functions of the API.

Question:{question}
API url:"""

MLB_URL_PROMPT = PromptTemplate(
    input_variables=[
        "api_docs",
        "question",
    ],
    template=MLB_PROMPT_TEMPLATE,
)


clarifai_llm = Clarifai(user_id=USER_ID,app_id=APP_ID, model_id=MODEL_ID, model_version_id=MODEL_VERSION_ID)

api_agent_chain = APIChain.from_llm_and_api_docs(clarifai_llm, baseball_api, verbose=True, api_url_prompt=MLB_URL_PROMPT)

api_agent_chain.run("What operations can be performed by this chatbot?")
    




> Entering new APIChain chain...
This is not a specific API call, but rather a question regarding the capabilities of the bot itself. As such, it doesn't have an API URL. It can be answered by examining the API documentation which shows that this bot can perform many operations like retrieve overall MLB team standings, individual player's batting and pitching statistics, a team batting statistics, top prospects, etc. for the specified season and team.


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:42                                                                                   │
│                                                                                                  │
│   39                                                                                             │
│   40 api_agent_chain = APIChain.from_llm_and_api_docs(clarifai_llm, baseball_api, verbose=Tru    │
│   41                                                                                             │
│ ❱ 42 api_agent_chain.run("What operations can be performed by this chatbot?")                    │
│   43                                                                                             │
│   44                                                                                             │
│                                                                                                  │
│ c:\Users\joefo\dev\mlb-stats-chatbot\mlb-env\lib\site-packages\langchain\chains\base.py:487 in   │
│ run                                                                                              │
│                                                                                                  │
│   484 │   │   if args and not kwargs:                                                            │
│   485 │   │   │   if len(args) != 1:                                                             │
│   486 │   │   │   │   raise ValueError("`run` supports only one positional argument.")           │
│ ❱ 487 │   │   │   return self(args[0], callbacks=callbacks, tags=tags, metadata=metadata)[       │
│   488 │   │   │   │   _output_key                                                                │
│   489 │   │   │   ]                                                                              │
│   490                                                                                            │
│                                                                                                  │
│ c:\Users\joefo\dev\mlb-stats-chatbot\mlb-env\lib\site-packages\langchain\chains\base.py:292 in   │
│ __call__                                                                                         │
│                                                                                                  │
│   289 │   │   │   )                                                                              │
│   290 │   │   except (KeyboardInterrupt, Exception) as e:                                        │
│   291 │   │   │   run_manager.on_chain_error(e)                                                  │
│ ❱ 292 │   │   │   raise e                                                                        │
│   293 │   │   run_manager.on_chain_end(outputs)                                                  │
│   294 │   │   final_outputs: Dict[str, Any] = self.prep_outputs(                                 │
│   295 │   │   │   inputs, outputs, return_only_outputs                                           │
│                                                                                                  │
│ c:\Users\joefo\dev\mlb-stats-chatbot\mlb-env\lib\site-packages\langchain\chains\base.py:286 in   │
│ __call__                                                                                         │
│                                                                                                  │
│   283 │   │   )                                                                                  │
│   284 │   │   try:                                                                               │
│   285 │   │   │   outputs = (                                                                    │
│ ❱ 286 │   │   │   │   self._call(inputs, run_manager=run_manager)                                │
│   287 │   │   │   │   if new_arg_supported                 

In [4]:
# Get the manifest
import requests
import yaml
f = requests.get('http://localhost:8000/.well-known/ai-plugin.json').text
manifest = yaml.load(f, Loader=yaml.Loader)

# Load the Clarifai wrapper GPT-4
from langchain.llms import Clarifai
from llama_index.llms import LangChainLLM
USER_ID = 'openai'
APP_ID = 'chat-completion'
MODEL_ID = 'GPT-4'
MODEL_VERSION_ID = 'ad16eda6ac054796bf9f348ab6733c72'
clarifai_llm = Clarifai(user_id=USER_ID,app_id=APP_ID, model_id=MODEL_ID, model_version_id=MODEL_VERSION_ID)

langchain_llm = LangChainLLM(llm=clarifai_llm)

from llama_hub.tools.chatgpt_plugin.base import ChatGPTPluginToolSpec
from llama_index.agent import OpenAIAgent
from llama_hub.tools.requests.base import RequestsToolSpec

requests_spec = RequestsToolSpec()
plugin_spec = ChatGPTPluginToolSpec(manifest)

# agent = OpenAIAgent.from_tools([*plugin_spec.to_tool_list(), *requests_spec.to_tool_list()], verbose=True)


{'operationId': 'getStandings', 'summary': 'Retrieves the overall standings for all teams across the MLB depending on the year that is supplied.', 'description': 'The operation will retrieve the ranking of every team within their respective division across Major League Baseball (MLB).', 'parameters': [{'in': 'query', 'name': 'year', 'required': True, 'schema': {'type': 'integer'}, 'description': 'Used to filter the year the overall standings should be returned for.'}]}
{'operationId': 'getNews', 'summary': 'Retrieves up to date news articles for teams from mlb.com', 'parameters': [{'in': 'query', 'name': 'team_name', 'required': True, 'schema': {'type': 'string', 'enum': ['whitesox', 'guardians', 'tigers', 'royals', 'twins', 'cubs', 'reds', 'brewers', 'pirates', 'cardinals', 'orioles', 'redsox', 'yankees', 'rays', 'bluejays', 'braves', 'marlins', 'mets', 'phillies', 'nationals', 'astros', 'angels', 'athletics', 'mariners', 'rangers', 'diamondbacks', 'rockies', 'dodgers', 'padres', 'gia

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:27                                                                                   │
│                                                                                                  │
│   24                                                                                             │
│   25 # agent = OpenAIAgent.from_tools([*plugin_spec.to_tool_list(), *requests_spec.to_tool_li    │
│   26                                                                                             │
│ ❱ 27 langchain_llm.chat("Hello!")                                                                │
│   28                                                                                             │
│                                                                                                  │
│ c:\Users\joefo\dev\mlb-stats-chatbot\mlb-env\lib\site-packages\llama_index\llms\base.py:151 in   │
│ wrapped_llm_chat                                                                                 │
│                                                                                                  │
│   148 │   │   │   │   │   │   EventPayload.SERIALIZED: _self.to_dict(),                          │
│   149 │   │   │   │   │   },                                                                     │
│   150 │   │   │   │   )                                                                          │
│ ❱ 151 │   │   │   │   f_return_val = f(_self, messages, **kwargs)                                │
│   152 │   │   │   │                                                                              │
│   153 │   │   │   │   if isinstance(f_return_val, Generator):                                    │
│   154 │   │   │   │   │   # intercept the generator and add a callback to the end                │
│                                                                                                  │
│ c:\Users\joefo\dev\mlb-stats-chatbot\mlb-env\lib\site-packages\llama_index\llms\langchain.py:56  │
│ in chat                                                                                          │
│                                                                                                  │
│    53 │                                                                                          │
│    54 │   @llm_chat_callback()                                                                   │
│    55 │   def chat(self, messages: Sequence[ChatMessage], **kwargs: Any) -> ChatResponse:        │
│ ❱  56 │   │   lc_messages = to_lc_messages(messages)                                             │
│    57 │   │   lc_message = self._llm.predict_messages(messages=lc_messages, **kwargs)            │
│    58 │   │   message = from_lc_messages([lc_message])[0]                                        │
│    59 │   │   return ChatResponse(message=message)                                               │
│                                                                                                  │
│ c:\Users\joefo\dev\mlb-stats-chatbot\mlb-env\lib\site-packages\llama_index\llms\langchain_utils. │
│ py:32 in to_lc_messages                                                                          │
│                                                                                                  │
│    29 def to_lc_messages(messages: Sequence[ChatMessage]) -> List[LCMessage]:                    │
│    30 │   lc_messages: List[LCMessage] = []                                                      │
│    31 │   for message in messages:                                                               │
│ ❱  32 │   │   if message.role == "user":                                                         │
│    33 │   │   │   lc_messages.append(                                                            │
│    34 │   │   │   │   HumanMessage(                        

In [18]:
# Implementing using 'custom agent with Plugin Retrieval'
from langchain.agents import (
    Tool,
    AgentExecutor,
    LLMSingleActionAgent,
    AgentOutputParser,
)
from langchain.prompts import StringPromptTemplate
from langchain import SerpAPIWrapper, LLMChain
from langchain.llms import Clarifai
from typing import List, Union
from langchain.schema import AgentAction, AgentFinish
from langchain.agents.agent_toolkits import NLAToolkit
from langchain.tools.plugin import AIPlugin
import re

#Initialize clarifai LLM
USER_ID = 'openai'
APP_ID = 'chat-completion'
MODEL_ID = 'GPT-4' 
MODEL_VERSION_ID = 'ad16eda6ac054796bf9f348ab6733c72'

clarifai_llm = Clarifai(user_id=USER_ID,app_id=APP_ID, model_id=MODEL_ID, model_version_id=MODEL_VERSION_ID)

PLUGIN = AIPlugin.from_url("http://localhost:8000/.well-known/ai-plugin.json")

from langchain.vectorstores import FAISS
from langchain.embeddings import ClarifaiEmbeddings
from langchain.schema import Document

EMBED_USER_ID = 'openai'
EMBED_APP_ID = 'embed'
EMBED_MODEL_ID = 'text-embedding-ada'
EMBED_MODEL_VERSION_ID = '7a55116e5fde47baa02ee5741039b149'

embeddings = ClarifaiEmbeddings(user_id=EMBED_USER_ID, app_id=EMBED_APP_ID, model_id=EMBED_MODEL_ID, model_version_id=EMBED_MODEL_VERSION_ID)

plugin = PLUGIN

docs = [
    Document(
        page_content=plugin.description_for_model,
        metadata={"plugin_name": plugin.name_for_model},
    )
]
vector_store = FAISS.from_documents(docs, embeddings)

toolkits_dict = {
    plugin.name_for_model: NLAToolkit.from_llm_and_ai_plugin(clarifai_llm, plugin)
}

retriever = vector_store.as_retriever()

def get_tools(query):
    # Get documents, which contain the Plugins to use
    docs = retriever.get_relevant_documents(query)
    # Get the toolkits, one for each plugin
    tool_kits = [toolkits_dict[d.metadata["plugin_name"]] for d in docs]
    # Get the tools: a separate NLAChain for each endpoint
    tools = []
    for tk in tool_kits:
        tools.extend(tk.nla_tools)
    return tools

template = """Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Action: the action to take, should be one of [{tool_names}]
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Final Answer: the final answer to the original input question

Begin!

Question: {input}
{agent_scratchpad}"""

from typing import Callable


# Set up a prompt template
class CustomPromptTemplate(StringPromptTemplate):
    # The template to use
    template: str
    ############## NEW ######################
    # The list of tools available
    tools_getter: Callable

    def format(self, **kwargs) -> str:
        # Get the intermediate steps (AgentAction, Observation tuples)
        # Format them in a particular way
        intermediate_steps = kwargs.pop("intermediate_steps")
        thoughts = ""
        for action, observation in intermediate_steps:
            thoughts += action.log
            thoughts += f"\nObservation: {observation}\nThought: "
        # Set the agent_scratchpad variable to that value
        kwargs["agent_scratchpad"] = thoughts
        ############## NEW ######################
        tools = self.tools_getter(kwargs["input"])
        # Create a tools variable from the list of tools provided
        kwargs["tools"] = "\n".join(
            [f"{tool.name}: {tool.description}" for tool in tools]
        )
        # Create a list of tool names for the tools provided
        kwargs["tool_names"] = ", ".join([tool.name for tool in tools])
        return self.template.format(**kwargs)

prompt = CustomPromptTemplate(
    template=template,
    tools_getter=get_tools,
    # This omits the `agent_scratchpad`, `tools`, and `tool_names` variables because those are generated dynamically
    # This includes the `intermediate_steps` variable because that is needed
    input_variables=["input", "intermediate_steps"],
)

class CustomOutputParser(AgentOutputParser):
    def parse(self, llm_output: str) -> Union[AgentAction, AgentFinish]:
        # Check if agent should finish
        if "Final Answer:" in llm_output:
            return AgentFinish(
                # Return values is generally always a dictionary with a single `output` key
                # It is not recommended to try anything else at the moment :)
                return_values={"output": llm_output.split("Final Answer:")[-1].strip()},
                log=llm_output,
            )
        # Parse out the action and action input
        regex = r"Action\s*\d*\s*:(.*?)\nAction\s*\d*\s*Input\s*\d*\s*:[\s]*(.*)"
        match = re.search(regex, llm_output, re.DOTALL)
        if not match:
            raise ValueError(f"Could not parse LLM output: `{llm_output}`")
        action = match.group(1).strip()
        action_input = match.group(2)
        # Return the action and action input
        return AgentAction(
            tool=action, tool_input=action_input.strip(" ").strip('"'), log=llm_output
        )

output_parser = CustomOutputParser()

# LLM chain consisting of the LLM and a prompt
llm_chain = LLMChain(llm=clarifai_llm, prompt=prompt)

tool_names = [tool.name for tool in tools]
agent = LLMSingleActionAgent(
    llm_chain=llm_chain,
    output_parser=output_parser,
    stop=["\nObservation:"],
    allowed_tools=tool_names,
)

agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent, tools=tools, verbose=True
)

agent_executor.run("What are the current MLB standings for the 2023 season?")






> Entering new AgentExecutor chain...
Action: MLB_Stats.getStandings
Action Input: {"season": 2023}

c:\Users\joefo\dev\mlb-stats-chatbot\mlb-env\lib\site-packages\langchain\chains\llm.py:278: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(




Observation:The API call to get the standings for the 2023 season encountered a '500: Internal Server Error'. This issue can commonly occur when there's a problem with the server. You may want to retry the request later, or if you're the developer, check the server logs for more detailed information.
The API encountered an error while trying to retrieve the 2023 season standings. Without this data, I can't provide the required information. 
Final Answer: I'm sorry, but there was a problem retrieving the MLB standings for the 2023 season. Please try again later or contact the administrator.

> Finished chain.


"I'm sorry, but there was a problem retrieving the MLB standings for the 2023 season. Please try again later or contact the administrator."